In [1]:
import os
import re

In [2]:
import numpy as np
from scipy.sparse.csgraph import shortest_path
import pandas as pd

In [3]:
pd.options.display.max_colwidth=500

In [4]:
ent_pattern = re.compile('T(\d+)\t([A-Z_]+) (\d+ \d+(;\d+ \d+)*)\t(.+)')
rel_pattern = re.compile('R(\d+)\t([A-Z_]+) Arg1:T(\d+) Arg2:T(\d+)')
ref_pattern = re.compile('R(\d+)\tCOREFERENCE Arg1:T(\d+) Arg2:T(\d+)')

folder = './test/sample/'

for filename in os.listdir(folder):
    if filename.endswith('.ann'):
        with open(folder + filename, 'r', encoding='utf8') as fin:
            text = fin.read()

        rels = rel_pattern.findall(text)
        if len(rels) == 0:
            continue
        assert len(rels) == int(rels[-1][0])

        df_ent = pd.DataFrame([(e[0], e[1], e[2], e[4]) for e in ent_pattern.findall(text)], 
                  columns=['id', 'type', 'range', 'text']).set_index('id')
        df_ref = pd.DataFrame(ref_pattern.findall(text), columns=['id', 'ent1', 'ent2'])
        df = df_ref.join(df_ent, on='ent1', rsuffix='1').join(df_ent, on='ent2', rsuffix='2')
        
        common_span = set(df['range']) & set(df['range2'])
        if len(common_span) != 0:
            print(filename)
            display(df)

            coref_dict = {r:e for e, r in zip(df['ent1'], df['range'])}
            ent_dict = {r:e for e, r in zip(df['ent2'], df['range2'])}

            all_spans = sorted(set(df['range']) | set(df['range2']))
            num_spans = len(all_spans)

            df_graph = pd.DataFrame(np.zeros((num_spans, num_spans), dtype=int), index=all_spans, columns=all_spans)

            for r1, r2 in zip(df['range'], df['range2']):
                df_graph.loc[r1, r2] = 1

            d = shortest_path(df_graph.values, directed=True)

            print('new rels')
            new_rels = []
            for e1, e2 in zip(*np.where(((~np.isinf(d)) & (d>1)))):
                r1 = all_spans[e1]
                r2 = all_spans[e2]
                new_rels.append((coref_dict[r1], ent_dict[r2]))

            with open(folder + filename, 'a', encoding='utf8') as fout:
                for i, (e1, e2) in enumerate(new_rels, start=len(rels)+1):
                    rel_str = 'R%d\tCOREFERENCE Arg1:T%s Arg2:T%s'%(i, e1, e2)
                    print(rel_str)
                    fout.write(rel_str + '\n')                        

0001.ann


,id,ent1,ent2,type,range,text,type2,range2,text2
0,11,18,1,COREFERENCE,701 802,"400 mg (88%) of [5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-acetic acid",ENTITY,6 91,"[5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-acetic acid"
1,12,13,12,COREFERENCE,830 843,a white solid,ENTITY,701 802,"400 mg (88%) of [5-(2,3-difluorophenyl)-3-methyl-2,4-dioxo-3,4-dihydro-2H-pyrimidin-1-yl]-acetic acid"


new rels
R13	COREFERENCE Arg1:T13 Arg2:T1
0002.ann


,id,ent1,ent2,type,range,text,type2,range2,text2
0,5,7,4,COREFERENCE,601 612,the solvent,ENTITY,367 395,ethanol (20 mL)/water (5 mL)
1,7,9,8,COREFERENCE,691 703,yellow solid,ENTITY,614 632;671 687,"The title compound (1.180 g, 95.2%)"
2,8,13,1,COREFERENCE,614 632;671 687,"The title compound (1.180 g, 95.2%)",ENTITY,22 139,"N-(3-chloro-4-fluorophenyl)-N-(2-fluoro-4-(hydrazinecarbonyl)benzyl)tetrahydro-2H-thiopyran-4-carboxamide 1,1-dioxide"


new rels
R9	COREFERENCE Arg1:T9 Arg2:T1
0003.ann


...

new rels
R14	COREFERENCE Arg1:T14 Arg2:T1
0046.ann


,id,ent1,ent2,type,range,text,type2,range2,text2
0,5,7,6,COREFERENCE,521 536,a white crystal,ENTITY,449 517,"racemic 3-aminoquinuclidine.diacetate (4.41 g: 39.0 mmol, yield=89%)"
1,6,11,1,COREFERENCE,449 517,"racemic 3-aminoquinuclidine.diacetate (4.41 g: 39.0 mmol, yield=89%)",ENTITY,36 73,racemic 3-aminoquinuclidine.diacetate


new rels
R7	COREFERENCE Arg1:T7 Arg2:T1
0049.ann


,id,ent1,ent2,type,range,text,type2,range2,text2
0,1,11,1,COREFERENCE,112 205,"7-(Cyclopropylmethyl)-1-(5-isopropylisoxazol-4-yl)methyl)-3-methyl-1H-purine-2,6(3H,7H)-dione",ENTITY,11 104,"7-(Cyclopropylmethyl)-1-(5-isopropylisoxazol-4-yl)methyl)-3-methyl-1H-purine-2,6(3H,7H)-dione"
1,9,10,2,COREFERENCE,638 761,"7-(cyclopropylmethyl)-1-(5-isopropylisoxazol-4-yl)methyl)-3-methyl-1H-purine-2,6(3H,7H)-dione (80.0 mg) with a yield of 37%",ENTITY,112 205,"7-(Cyclopropylmethyl)-1-(5-isopropylisoxazol-4-yl)methyl)-3-methyl-1H-purine-2,6(3H,7H)-dione"


new rels
R10	COREFERENCE Arg1:T10 Arg2:T1
